In [1]:
%pip install --upgrade --quiet  amadeus > /dev/null


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.4.0_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -qU langchain-community


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.4.0_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
from langchain_community.agent_toolkits.amadeus.toolkit import AmadeusToolkit
from amadeus import Client

os.environ["AMADEUS_CLIENT_ID"] = "<YOUR_CLIENT_ID>"  # Replace with your Amadeus client ID
os.environ["AMADEUS_CLIENT_SECRET"] = "<YOUR_CLIENT_SECRET>"  # Replace with your Amadeus client secret
os.environ["OPENAI_API_KEY"] = "<YOUR_KEY_HERE>"  # Replace with your OpenAI API key
os.environ["AMADEUS_HOSTNAME"] = "test"

print("Rebuilding Pydantic models...")
from langchain_community.tools.amadeus.closest_airport import AmadeusClosestAirport

# Force rebuild the models with the client available
AmadeusClosestAirport.model_rebuild()
print("ClosestAirport Models rebuilt successfully")

toolkit = AmadeusToolkit()
tools = toolkit.get_tools()

print(tools)


Rebuilding Pydantic models...
ClosestAirport Models rebuilt successfully
[AmadeusClosestAirport(client=<amadeus.amadeus.Client object at 0x1162b0b50>, llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x1160458c0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x116046140>, temperature=0.0, model_kwargs={}, openai_api_key='<YOUR_KEY_HERE>', openai_proxy='')), AmadeusFlightSearch(client=<amadeus.amadeus.Client object at 0x1162b0c50>)]


In [13]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser
from langchain.tools.render import render_text_description_and_args
from langchain_community.chat_models import ChatOpenAI

In [14]:
llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0)

prompt = hub.pull("hwchase17/react-json")
agent = create_react_agent(
    llm,
    tools,
    prompt,
    tools_renderer=render_text_description_and_args,
    output_parser=ReActJsonSingleInputOutputParser(),
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

/opt/homebrew/Cellar/jupyterlab/4.4.0_1/libexec/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [11]:
agent_executor.invoke({"input": "What is the name of the airport in Cali, Colombia?"})



> Entering new AgentExecutor chain...
Question: What is the name of the airport in Cali, Colombia?
Thought: We can use the closest_airport tool to find the airport in Cali, Colombia.
Action:
```
{
  "action": "closest_airport",
  "action_input": {"location": "Cali, Colombia"}
}
```content='{\n  "iataCode": "CLO"\n}' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 57, 'total_tokens': 69, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run--5b95c162-be5e-4180-8277-8a502bf72d68-0'Final Answer: The airport in Cali, Colombia is named Alfonso Bonilla Aragón International Airport with the IATA code "CLO".

> Finished chain.


{'input': 'What is the name of the airport in Cali, Colombia?',
 'output': 'The airport in Cali, Colombia is named Alfonso Bonilla Aragón International Airport with the IATA code "CLO".'}